In [ ]:
!pip install textblob


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import numpy as np
import textblob
from textblob import TextBlob
import pickle
import import_ipynb


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

from wordcloud import WordCloud

import unidecode
import nltk
nltk.download('punkt')

import num2words

from autocorrect import Speller
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import contractions
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### Texte en minuscule
def lower_case_convertion(text):
	lower_text = text.lower()
	return lower_text

### Suppression des tags HTML
def remove_html_tags(text):
	html_pattern = r'<.*?>'
	without_html = re.sub(pattern=html_pattern, repl=' ', string=text)
	return without_html

# Suppression des URLS
def remove_urls(text):
	url_pattern = r'https?://\S+|www\.\S+'
	without_urls = re.sub(pattern=url_pattern, repl=' ', string=text)
	return without_urls

#ASCII
def accented_to_ascii(text):

	# apply unidecode function on text to convert
	# accented characters to ASCII values
	text = unidecode.unidecode(text)
	return text

### Contraction
def expand_contractions(text):
  ### We can use the dict : https://stackoverflow.com/questions/60901735/importerror-cannot-import-name-contraction-map-from-contractions
	return contractions.fix(text)


#######################################################################################################################################

### Suppresion de la ponctuation 
import string

def remove_punctuation(text) :
  return text.translate(str.maketrans('', '', string.punctuation))


In [ ]:
def preprocessing(text) :
  text = lower_case_convertion(text)
  text = remove_html_tags(text)
  text = remove_urls(text)
  text = accented_to_ascii(text)
  text =  expand_contractions(text)
  text = remove_punctuation(text)
  return text 

In [ ]:
def tag_words(sentence_tokenized):

    keywordSet = {"never", "nothing", "nowhere", "none", "not"}

    for word in sentence_tokenized:
      if (word in keywordSet) and (sentence_tokenized.index(word) < len(sentence_tokenized)-1):
        sentence_tokenized[sentence_tokenized.index(word)+1]=sentence_tokenized[sentence_tokenized.index(word)+1] + '_NEG'
        sentence_tokenized.pop(sentence_tokenized.index(word))
    return sentence_tokenized



In [ ]:
nltk.download('stopwords')
def remove_stop_word(text) :
  stop_words = stopwords.words('english')
  tokens = word_tokenize(text)
  tokens_without_sw = [word for word in tokens if not word in stop_words]
  text = ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')

def merge(dict1, dict2):
    for i in dict2.keys():
        dict1[i]=dict2[i]
    return dict1
    
def pos_tagging(text):
  ### POS_TAGGING
  lmtzr = WordNetLemmatizer()
  all_tag_words = {}
  tokens = word_tokenize(text)
  tagged = dict(nltk.pos_tag(tokens))
  all_tag_words = merge(tagged, all_tag_words)
  return all_tag_words

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

# initialize lemmatizer object
lemma = WordNetLemmatizer()  
### Lemmatize with POS tagging 
def lemmatize(text, all_tag_words):
  tokens = ''.join(text).split()
  for i,token in enumerate(tokens) :
    try :
      tokens[i] = lemma.lemmatize(token,get_wordnet_pos(all_tag_words[token]))
    except : 
      pass
  text = ' '.join(tokens)
  return text


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
labels_dict = {
    0 : "bad customer service at phone",
    1 : "disappointed taste",
    2 : "cold pizza",
    3 : "not enough chicken",
    4 : "bad food quality",
    5 : "bad service",
    6 : "bad burgers",
    7 : "long wait",
    8 : "bad experience on all levels",
    9 : "bad experience at the bar",
    10 : "problem with the delivery or the online order",
    11 : "bad experience several times",
    12 : "unorganized staff",
    13 : "poor quality sushis",
    14 : "dangerous place"
}

In [ ]:
with open('/content/drive/MyDrive/Sitou/model_entraîné_InesArif.pickle', 'rb') as f:
    model = pickle.load(f) 

with open('/content/drive/MyDrive/Sitou/vectorizer_InesArif.pickle', 'rb') as f:
    vectorizer = pickle.load(f) 


In [ ]:
def predict(text : str,nb_features:int,labels_dict=labels_dict, blob=True) :

  if blob == True :
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    print(polarity)
    if polarity > 0 :
      return polarity, None

    if polarity<0 :
      text = preprocessing(text)
      dict_pos = pos_tagging(text)
      text = lemmatize(text, dict_pos)
      text = [text]
      vect = vectorizer.transform(text)      
      nmf_features = list(list(model.transform(vect))[0])
      nmf_features_copy = nmf_features
      indexes = []
      indexes.append(nmf_features.index(max(nmf_features_copy)))
      nmf_features_copy.pop(indexes[0])
      
      for i in range(nb_features-1):
        
        index = nmf_features.index(max(nmf_features_copy))
        indexes.append(index)
        nmf_features_copy.pop(index)

      for i in indexes :
        print(labels_dict[i])
      
      topics = []
      for i in indexes:
        topics.append(labels_dict[i])
      return polarity, topics